In [14]:
import math
import random
import skimage.io as io
import matplotlib.pyplot as plt
import keras
import pickle
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import sys
# import skvideo.io
import json
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import glob
import re
from collections import defaultdict

import nltk
from nltk.corpus import cmudict
import tensorflow as tf
import wer

In [15]:
!nvidia-smi

Tue May 14 04:54:29 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.48                 Driver Version: 410.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3E:00.0 Off |                  N/A |
| 31%   54C    P2   115W / 250W |  10743MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:3F:00.0 Off |                  N/A |
| 29%   28C    P8    20W / 250W |  10743MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [16]:
os.environ['CUDA_VISIBLE_DEVICES']='3'

In [17]:
phoneme_list = [] 
phoneme_dict = {}

with open("/n/fs/scratch/jiaqis/cmudict-master/cmudict.phones", 'r') as fp:
    i = 0
    line = fp.readline()
    while line:
        phoneme = line.split()[0].strip()
        phoneme_property = line.split()[1].strip()
        phoneme_list.append((phoneme, phoneme_property))
        phoneme_dict[phoneme] = i+3
        line = fp.readline()
        i=i+1
        
phoneme_dict['START'] = 0
phoneme_dict["END"] = 1
phoneme_dict["UNK"] = 2
print(phoneme_list, phoneme_dict)

([('AA', 'vowel'), ('AE', 'vowel'), ('AH', 'vowel'), ('AO', 'vowel'), ('AW', 'vowel'), ('AY', 'vowel'), ('B', 'stop'), ('CH', 'affricate'), ('D', 'stop'), ('DH', 'fricative'), ('EH', 'vowel'), ('ER', 'vowel'), ('EY', 'vowel'), ('F', 'fricative'), ('G', 'stop'), ('HH', 'aspirate'), ('IH', 'vowel'), ('IY', 'vowel'), ('JH', 'affricate'), ('K', 'stop'), ('L', 'liquid'), ('M', 'nasal'), ('N', 'nasal'), ('NG', 'nasal'), ('OW', 'vowel'), ('OY', 'vowel'), ('P', 'stop'), ('R', 'liquid'), ('S', 'fricative'), ('SH', 'fricative'), ('T', 'stop'), ('TH', 'fricative'), ('UH', 'vowel'), ('UW', 'vowel'), ('V', 'fricative'), ('W', 'semivowel'), ('Y', 'semivowel'), ('Z', 'fricative'), ('ZH', 'fricative')], {'IY': 20, 'START': 0, 'W': 38, 'DH': 12, 'Y': 39, 'HH': 18, 'CH': 10, 'JH': 21, 'ZH': 41, 'END': 1, 'EH': 13, 'UNK': 2, 'NG': 26, 'TH': 34, 'AA': 3, 'B': 9, 'AE': 4, 'D': 11, 'G': 17, 'F': 16, 'AH': 5, 'K': 22, 'M': 24, 'L': 23, 'AO': 6, 'N': 25, 'IH': 19, 'S': 31, 'R': 30, 'EY': 15, 'T': 33, 'AW': 7,

In [18]:
pron_dict = cmudict.dict()

In [19]:
DATA_DIR = "/n/fs/scratch/jiaqis/LRS3-TED/"
SAVE_DIR = "/n/fs/scratch/jiaqis/LRS3-TED-Extracted/"

In [20]:
test_ID_list = json.load(open('test_ID_list.json', "r"))
trainval_ID_list = json.load(open('trainval_ID_list.json', "r"))

In [21]:
import logging

import numpy as np
import tensorflow as tf
from tensorflow.contrib import layers


GO_TOKEN = 0
END_TOKEN = 1
UNK_TOKEN = 2

In [22]:
def frame_wise_op(inputs, operation, **kwargs):
    # batch x timesteps x w x h x c
    # or
    # batch x timesteps x c
    inputs_flat = tf.reshape(inputs, tf.concat([tf.constant(-1, shape=(1,)), 
                                                tf.shape(inputs)[2:]], axis=0))
    print(inputs, inputs_flat)
    outputs_flat = operation(inputs_flat, **kwargs)

    output_shape = tf.concat([tf.shape(inputs)[:2], tf.shape(outputs_flat)[1:]], 0)
    outputs = tf.reshape(outputs_flat, output_shape)

    return outputs

In [23]:
def seq2seq(mode, features, labels, params):
    is_training = True
    
    vocab_size = params['vocab_size']
    embed_dim = params['embed_dim']
    num_units = params['num_units']
    input_max_length = params['input_max_length']
    output_max_length = params['output_max_length']

    inp = features['input']
    output = features['output']
    batch_size = tf.shape(inp)[0]
    start_tokens = tf.zeros([batch_size], dtype=tf.int64)
    print(inp, start_tokens)
    train_output = tf.concat([tf.expand_dims(start_tokens, 1), output], 1)
    # b x f x w x h x c
    input_lengths = tf.reduce_sum(tf.to_int32(tf.reduce_sum(tf.to_int32(tf.not_equal(inp, 0.0)), 
                                                            axis=(2,3,4))>0), 1)
    # b x seq
    output_lengths = tf.reduce_sum(tf.to_int32(tf.not_equal(train_output, END_TOKEN)), 1)
    print(train_output, input_lengths, output_lengths, output)

    with tf.variable_scope('video_embed', reuse=tf.AUTO_REUSE):
        # 224 x 224 x 64
        conv1 = frame_wise_op(inp,
                     operation=tf.layers.conv2d,
                     filters=64,
                     kernel_size=[3, 3],
                     padding='same')
#         conv1 = tf.layers.batch_normalization(
#                     conv1,
#                     axis=-1,
#                     training=is_training,
#                     trainable=True
#                 )
        conv1 = tf.nn.relu(conv1)

        # 112 x 112 x 64
        down1 = frame_wise_op(conv1,
                            operation=tf.layers.max_pooling2d,
                            pool_size=[2, 2],
                            strides=2)
#         down1 = tf.layers.max_pooling3d(conv1,
#                             pool_size=[2, 2, 2],
#                             strides=2)
        # 112 x 112 x 128
        conv2 = frame_wise_op(down1,
                     operation=tf.layers.conv2d,
                     filters=128,
                     kernel_size=[3, 3],
                     padding='same',)
#         conv2 = tf.layers.batch_normalization(
#             conv2,
#             axis=-1,
#             training=is_training,
#             trainable=True
#         )
        conv2 = tf.nn.relu(conv2)
        
        # 56 x 56 x 128
        down2 = frame_wise_op(conv2,
                            operation=tf.layers.max_pooling2d,
                            pool_size=[2, 2],
                            strides=2)
        # 56 x 56 x 256
        conv3 = frame_wise_op(down2,
                     operation=tf.layers.conv2d,
                     filters=256,
                     kernel_size=[3, 3],
                     padding='same')
#         conv3 = tf.layers.batch_normalization(
#             conv3,
#             axis=-1,
#             training=is_training,
#             trainable=True
#         )
        conv3 = tf.nn.relu(conv3)
        
        # 28 x 28 x 256
        down3 = frame_wise_op(conv3,
                            operation=tf.layers.max_pooling2d,
                            pool_size=[2, 2],
                            strides=2)
        # 28 x 28 x 256
        conv4 = frame_wise_op(down3,
                     operation=tf.layers.conv2d,
                     filters=256,
                     kernel_size=[3, 3],
                     padding='same')
#         conv4 = tf.layers.batch_normalization(
#             conv4,
#             axis=-1,
#             training=is_training,
#             trainable=True
#         )
        conv4 = tf.nn.relu(conv4)
        
        # 14 x 14 x 256
        down4 = frame_wise_op(conv4,
                            operation=tf.layers.max_pooling2d,
                            pool_size=[2, 2],
                            strides=2)
        # 14 x 14 x 256
        conv5 = frame_wise_op(down4,
                     operation=tf.layers.conv2d,
                     filters=256,
                     kernel_size=[3, 3],
                     padding='same')
#         conv5 = tf.layers.batch_normalization(
#             conv5,
#             axis=-1,
#             training=is_training,
#             trainable=True
#         )
        conv5 = tf.nn.relu(conv5)
        
        # 7 x 7 x 256
        down5 = frame_wise_op(conv5,
                            operation=tf.layers.max_pooling2d,
                            pool_size=[2, 2],
                            strides=2)
        # 256
        down5_shape =down5.shape.as_list()
        flattened = frame_wise_op(down5, 
                           operation=tf.reshape,
                           shape=(-1, down5_shape[-1]*down5_shape[-2]*down5_shape[-3]))
        fc6 = frame_wise_op(flattened, 
                           operation=tf.layers.dense,
                           units=256,
                           activation='relu')
    
    output_embed = layers.embed_sequence(
        train_output, vocab_size=vocab_size, embed_dim=embed_dim, scope='embed')
    with tf.variable_scope('embed', reuse=True):
        embeddings = tf.get_variable('embeddings')

    cell = tf.contrib.rnn.GRUCell(num_units=num_units)
    encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(cell, fc6, dtype=tf.float32)
    print(encoder_outputs)

    train_helper = tf.contrib.seq2seq.TrainingHelper(output_embed, output_lengths)
#     train_helper = tf.contrib.seq2seq.ScheduledEmbeddingTrainingHelper(
#         output_embed, output_lengths, embeddings, 0.7
#     )
    pred_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        embeddings, start_tokens=tf.to_int32(start_tokens), end_token=1)

    def decode(helper, scope, reuse=None):
        with tf.variable_scope(scope, reuse=reuse):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units=num_units, memory=encoder_outputs,
                memory_sequence_length=input_lengths)
            cell = tf.contrib.rnn.GRUCell(num_units=num_units)
            attn_cell = tf.contrib.seq2seq.AttentionWrapper(
                cell, attention_mechanism, attention_layer_size=num_units / 2)
            out_cell = tf.contrib.rnn.OutputProjectionWrapper(
                attn_cell, vocab_size, reuse=reuse
            )
            decoder = tf.contrib.seq2seq.BasicDecoder(
                cell=out_cell, helper=helper,
                initial_state=out_cell.zero_state(
                    dtype=tf.float32, batch_size=batch_size))
                #initial_state=encoder_final_state)
            outputs = tf.contrib.seq2seq.dynamic_decode(
                decoder=decoder, output_time_major=False,
                impute_finished=True, maximum_iterations=output_max_length
            )
            return outputs[0]
    train_outputs = decode(train_helper, 'decode')
    print(train_outputs, train_outputs.rnn_output)
    pred_outputs = decode(pred_helper, 'decode', reuse=True)

    tf.identity(train_outputs.sample_id[0], name='train_pred')
    weights = tf.to_float(tf.not_equal(train_output[:, :-1], 1))
    loss = tf.contrib.seq2seq.sequence_loss(
        train_outputs.rnn_output, output, weights=weights)
    train_op = layers.optimize_loss(
        loss, tf.train.get_global_step(),
        optimizer=params.get('optimizer', 'Adam'),
        learning_rate=params.get('learning_rate', 0.005),
        learning_rate_decay_fn = (lambda p1, p2: tf.train.exponential_decay(
                                                p1,
                                                p2,
                                                1000,
                                                0.9,
                                                staircase=False,
                                            )),
        summaries=['loss', 'learning_rate'])

    tf.identity(pred_outputs.sample_id[0], name='predictions')
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions={'train_pred':train_outputs.sample_id, 'predictions':pred_outputs.sample_id},
        loss=loss,
        train_op=train_op
    )


def tokenize_and_map(line, vocab):
    return [vocab.get(token, UNK_TOKEN) for token in line.split(' ')]

def make_input_fn(
        data_dir, subset, list_IDs, prons, phonemes,
        input_max_length, output_max_length,
        img_size, keypoint_img_size,
        keypoint_size,
        batch_size=1,
        input_process=tokenize_and_map, 
        output_process=tokenize_and_map):

    def input_fn():
        inp = tf.placeholder(tf.float32, shape=[None, None, img_size[0], img_size[1], img_size[2]+keypoint_size], name='input')
        output = tf.placeholder(tf.int64, shape=[None, None], name='output')
        tf.identity(inp[0], 'input_0')
        tf.identity(output[0], 'output_0')
        return {
            'input': inp,
            'output': output,
        }, None

    def sampler():
        while True:
            rnd = random.randint(0, len(list_IDs)-1)
            v_ID = list_IDs[rnd]
            v_url, v_index = v_ID
            filepath = os.path.join(data_dir, subset, v_url, v_index)
            v_V, v_F, v_T = prepare_data(filepath, img_size, keypoint_img_size,
                                           keypoint_size)
            num_frames = v_V.shape[0]
            v = np.concatenate([v_V, v_F], axis=-1)
            yield {
                    'input':  v[:min(num_frames,input_max_length), :, :, :], # v_F[:input_max_length]),
                    'output': v_T[:output_max_length - 1] + [END_TOKEN]
                }

    sample_me = sampler()

    def feed_fn():
        inputs, outputs = [], []
        input_length, output_length = 0, 0
        for i in range(batch_size):
            rec = sample_me.next()
            inputs.append(rec['input'])
            outputs.append(rec['output'])
            num_frames = inputs[-1].shape[0] #
#             print(inputs[-1].shape)
            input_length = max(input_length, num_frames)
            output_length = max(output_length, len(outputs[-1]))
        # Pad me right with </S> token.
        for i in range(batch_size):
            num_frames = inputs[i].shape[0]
            if input_length>num_frames:
                new_input_V = np.pad(inputs[i], [(input_length - num_frames, 0), (0,0), (0,0), (0,0)], 'constant')
                # new_input_F = np.pad(inputs[i][1], [output_length - num_frames, 0, 0, 0], 'constant')
                inputs[i] = new_input_V
            outputs[i] += [END_TOKEN] * (output_length - len(outputs[i]))
        return {
            'input:0': inputs,
            'output:0': outputs
        }

    return input_fn, feed_fn


def get_rev_vocab(vocab):
    return {idx: key for key, idx in vocab.iteritems()}

def get_formatter(keys, vocab):
    rev_vocab = get_rev_vocab(vocab)
    def to_str(sequence):
        tokens = [
            rev_vocab.get(x, "<UNK>") for x in sequence]
        return ' '.join(tokens)

    def format(values):
        res = []
        for key in keys:
            res.append("%s = %s" % (key, to_str(values[key])))
        return '\n'.join(res)
    return format

def get_editdist_formatter(keys, vocab):
    rev_vocab = get_rev_vocab(vocab)
    
    def to_str(sequence):
        tokens = [
            rev_vocab.get(x, "<UNK>") for x in sequence]
        return tokens

    def format(values):
        labels = values[keys[0]]  
        clean_labels = []
        for label in labels:
            if label == END_TOKEN:
                break
            else:
                clean_labels.append(label)
        clean_labels = to_str(clean_labels)
        
        preds =  values[keys[1]]
        clean_preds = []
        for pred in preds:
            if pred == END_TOKEN:
                break
            else:
                clean_preds.append(pred)
        clean_preds = to_str(clean_preds)
        wer.wer(clean_labels, clean_preds)
        return '\n====== WER '+ keys[1]+' ======\n'
    
    return format
        
def train_seq2seq(data_dir, subset, model_dir, list_IDs, prons, phonemes):
    params = {
        'vocab_size': 42,
        'batch_size': 2,
        'input_max_length': 100,
        'output_max_length': 100,
        'embed_dim': 100,
        'num_units': 256,
        'img_size': (224, 224, 3), 
        'keypoint_img_size': (224, 224),
        'keypoint_size': 20,
        'n_classes':39,
        'num_tokens': 42
    }
    est = tf.estimator.Estimator(
        model_fn=seq2seq,
        model_dir=model_dir, params=params)

    input_fn, feed_fn = make_input_fn(
                            data_dir, subset, list_IDs, prons, phonemes,
                            params['input_max_length'], params['output_max_length'],
                            params['img_size'], params['keypoint_img_size'],
                            params['keypoint_size'],
                            params['batch_size'])

    # Make hooks to print examples of inputs/predictions.
    print_inputs = tf.train.LoggingTensorHook(
        ['output_0'], every_n_iter=100,
        formatter=get_formatter(['output_0'], phonemes))
    print_predictions = tf.train.LoggingTensorHook(
        ['predictions', 'train_pred'], every_n_iter=100,
        formatter=get_formatter(['predictions', 'train_pred'], phonemes))
    print_train_edit_distance = tf.train.LoggingTensorHook(
        ['output_0', 'train_pred'], every_n_iter=100,
        formatter=get_editdist_formatter(['output_0', 'train_pred'], phonemes))
    print_pred_edit_distance = tf.train.LoggingTensorHook(
        ['output_0', 'predictions'], every_n_iter=100,
        formatter=get_editdist_formatter(['output_0', 'predictions'], phonemes))

    est.train(
        input_fn=input_fn,
        hooks=[tf.train.FeedFnHook(feed_fn), print_inputs, print_predictions, 
                                             print_train_edit_distance, print_pred_edit_distance],
        steps=200000)

In [28]:
def prepare_data(filepath, img_size, keypoint_img_size, keypoint_size):
    # images
    # frames x rows x cols x channels
    visual_cube = []
    # keypoint features
    feature_cube = []
    features = json.load(open(filepath + ".json", 'r'))
    # Target Text/phonemes
    labels = []
    text = open(filepath+".txt", 'r').readline()
    words = text[5:].lower().strip().split()
    for word in words:
        word_phonemes = pron_dict[word][0]
        word_indices = [phoneme_dict[clean_pron(phon)] for phon in word_phonemes]
        labels.extend(word_indices)
        
    acc = 0
    for imgFilename in sorted(glob.glob(filepath + "_*.jpg")): # 
        if 'mouth' in imgFilename: # 
            continue               #
        x = image.img_to_array(
              image.load_img(imgFilename, target_size=img_size))/255.0
#         x = np.expand_dims(x, axis=0)
#         x = preprocess_input(x)
        visual_cube.append(x)
        
        mask = np.zeros((keypoint_img_size[0], keypoint_img_size[1], keypoint_size))
        framenum = str(int(imgFilename.split("_")[-1].split(".")[0])) # 
        f_feature = features[framenum]['mouthCoords']
        for ft_index in range(keypoint_size):
            # TODO: check range of outputs
            keypoint_x = min(f_feature[ft_index][0] - 1, 223)
            keypoint_y = min(f_feature[ft_index][1] - 1, 223)
            mask[keypoint_y, keypoint_x, ft_index] = 1.0
        feature_cube.append(mask)
        acc+=1
    return np.array(visual_cube), np.array(feature_cube), labels

In [29]:
def clean_pron(pron):
    """Remove stress from pronunciations."""
    return re.sub(r"\d", "", pron)

def make_triphones(pron):
    """Output triphones from a word's pronunciation."""
    if len(pron) < 3:
        return []
    # Junk on end is to make word boundaries work
    return ([((pron[idx - 2], pron[idx - 1]), pron[idx])
             for idx in range(2, len(pron))] + [(('#', '#'), pron[0])] +
            [((pron[-2], pron[-1]), '#')])
                                                
def triphone_probs(prons):
    """Calculate triphone probabilities for pronunciations."""
    context_counts = defaultdict(lambda: defaultdict(int))
    for pron in prons:
        for (context, phoneme) in make_triphones(pron):
            context_counts[context][phoneme] += 1
            
    for (context, outcomes) in context_counts.items():
        total_outcomes = sum(outcomes.values())
        for outcome, count in outcomes.items():
            context_counts[context][outcome] = float(count) / total_outcomes
        
    return context_counts

In [30]:
def main():
#     tf.logging.logger.setLevel(logging.INFO)
    train_seq2seq(SAVE_DIR, 'trainval', './model/seq2seq_VF_224_bn', trainval_ID_list, pron_dict, phoneme_dict)

In [ ]:
main()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efbbcaf6210>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './model/seq2seq_VF_224_bn', '_train_distribute': None, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
(<tf.Tensor 'input:0' shape=(?, ?, 224, 224, 23) dtype=float32>, <tf.Tensor 'zeros:0' shape=(?,) dtype=int64>)
(<tf.Tensor 'concat:0' shape=(?, ?) dtype=int64>, <tf.Tensor 'Sum_1:0' shape=(?,) dtype=int32>, <tf.Tensor 'Sum_2:0' shape=(?,) dtype=int32>, <tf.Tensor 'output:0' sh

INFO:tensorflow:loss = 2.1782942, step = 38876 (356.743 sec)
INFO:tensorflow:global_step/sec: 0.284402
INFO:tensorflow:output_0 = IH T G IH V Z DH EH M S T R AH K CH ER END
INFO:tensorflow:predictions = AH N D AY S EH D W AH T END
train_pred = AH T S OW V Y AH AH M F AH AA IH K AH AH AH
REF: IH T G IH V Z DH EH M S T  R  AH K CH ER   
HYP: AH T S OW V Y AH AH M F AH AA IH K AH AH AH
EVA: S    S S    S S  S    S S  S  S    S  S  I 
WER: 81.25%
INFO:tensorflow:
====== WER train_pred ======

REF: IH T G  IH V Z  DH EH M S T R AH K CH ER
HYP:      AH N  D AY S  EH     D W AH      T 
EVA: D  D S  S  S S  S     D D S S    D D  S 
WER: 87.50%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 1.864342, step = 38976 (351.633 sec)
INFO:tensorflow:Saving checkpoints for 39023 into ./model/seq2seq_VF_224_bn/model.ckpt.
INFO:tensorflow:global_step/sec: 0.281846
INFO:tensorflow:output_0 = S OW DH IH S G OW Z B AE K T UW M AY F ER S T K W EH S CH AH N END
INFO:tensorflow:predicti

REF: AH N D DH EH N  W AH T D UW Y UW D  UW
HYP: AH N D       AY W        AA Z R  AY T 
EVA:        D  D  S    D  D D S  S S  S  S 
WER: 73.33%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 1.7176894, step = 39876 (355.744 sec)
INFO:tensorflow:Saving checkpoints for 39909 into ./model/seq2seq_VF_224_bn/model.ckpt.
INFO:tensorflow:global_step/sec: 0.281406
INFO:tensorflow:output_0 = DH EY AO L W AA N T T UW T R AY IH T AA N END END END END END END END
INFO:tensorflow:predictions = AH N D AY W AA Z R AY T END
train_pred = AH AE W L N EY Z T S UW S EH IY T T END N DH START START START START START START
REF: DH EY AO L W AA N T T UW T R AY IH T AA N
HYP: AH AE W  L N EY Z T S UW   S EH IY T    T
EVA: S  S  S    S S  S   S    D S S  S    D  S
WER: 76.47%
INFO:tensorflow:
====== WER train_pred ======

REF: DH EY AO L  W AA N T T UW T R AY IH T AA N
HYP: AH N  D  AY W AA          Z R AY    T     
EVA: S  S  S  S       D D D D  S      D    D  D
WER: 70.59%
INFO:tensorf

INFO:tensorflow:loss = 1.6736335, step = 40876 (293.909 sec)
INFO:tensorflow:Saving checkpoints for 40935 into ./model/seq2seq_VF_224_bn/model.ckpt.
INFO:tensorflow:global_step/sec: 0.335785
INFO:tensorflow:output_0 = AY W AA Z L AY K AO L R AY T END END END END END END END
INFO:tensorflow:predictions = DH AE T S W AH T AY W AA N T T UW T AO K IH NG AH B AW T END
train_pred = DH L AA Z V AY K AY L DH AY T END START START START START START START
REF: AY W AA Z L AY K AO L R  AY T
HYP: DH L AA Z V AY K AY L DH AY T
EVA: S  S      S      S    S      
WER: 41.67%
INFO:tensorflow:
====== WER train_pred ======

REF:                  AY W AA Z L AY K    AO L R  AY         T
HYP: DH AE T S W AH T AY W AA N T T  UW T AO K IH NG AH B AW T
EVA: I  I  I I I I  I         S S S  S  I    S S  S  I  I I   
WER: 150.00%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 1.4178605, step = 40976 (297.766 sec)
INFO:tensorflow:global_step/sec: 0.333457
INFO:tensorflow:output_0 = AH N D 

REF: AY D  IH D    AH N T      L AY K IH      T
HYP: W  IY HH AE V AH N T R IH L IY N OW W AH T
EVA: S  S  S  S  I        I I    S  S S  I I   
WER: 100.00%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 2.018447, step = 41876 (337.922 sec)
INFO:tensorflow:Saving checkpoints for 41884 into ./model/seq2seq_VF_224_bn/model.ckpt.
INFO:tensorflow:global_step/sec: 0.287413
INFO:tensorflow:output_0 = W AH T D IH D Y UW M EY K AH V IH T END
INFO:tensorflow:predictions = W IY HH AE V AH N T R IH L IY N OW W AH T END
train_pred = W IY T IH UW D DH UW D AY K END B IH T END
REF: W AH T D IH    D Y  UW M EY K AH V IH T
HYP: W IY T   IH UW D DH UW D AY K          
EVA:   S    D    I    S     S S    D  D D  D
WER: 66.67%
INFO:tensorflow:
====== WER train_pred ======

REF: W            AH   T D IH D Y UW M EY K AH V IH T
HYP: W IY HH AE V AH N T R IH   L IY N OW W AH      T
EVA:   I  I  I  I    I   S    D S S  S S  S    D D   
WER: 93.33%
INFO:tensorflow:
====== WER prediction

REF: IH T S N AA T G  OW IH NG T UW CH EY N JH OW V ER N  AY T
HYP: W  T S N AA T AH OW IH NG T UW D  EY N JH AH Z ER TH AH T
EVA: S             S                S          S  S    S  S   
WER: 31.82%
INFO:tensorflow:
====== WER train_pred ======

REF: IH T S N AA T G  OW IH NG T UW CH EY N JH OW V ER N  AY T
HYP:             W IY HH AE V  T UW               D UW DH AE T
EVA: D  D D D D  S S  S  S  S       D  D  D D  D  S S  S  S   
WER: 86.36%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 2.27086, step = 43076 (320.196 sec)
INFO:tensorflow:global_step/sec: 0.311903
INFO:tensorflow:output_0 = AY M G OW IH NG T UW M EY K AH B EH T ER HH EH D L AY T END
INFO:tensorflow:predictions = DH AE T S W AH T AY W AA N T T UW T UW M AY T AH B AW T END
train_pred = DH W G OW IH NG T UW G EY K IH B AW T ER END AE R F AY K END
REF: AY M G OW IH NG T UW M EY K AH B EH T ER HH EH D L AY T
HYP: DH W G OW IH NG T UW G EY K IH B AW T ER               
EVA: S  S                 S  

INFO:tensorflow:global_step/sec: 0.3534
INFO:tensorflow:output_0 = Y UW W AA N T T UW K IY P P IY P AH L AO F Y AO R B AE K END
INFO:tensorflow:predictions = DH AE T S W AH T AY W AA N T T UW T UW M AY T AH B AW T END
train_pred = DH UW N AA N T T UW S AY P F L P AH L END L DH UW R S AE K END
REF: Y  UW W AA N T T UW K IY P P IY P AH L AO F Y AO R B AE K
HYP: DH UW N AA N T T UW S AY P F L  P AH L                   
EVA: S     S             S S    S S         D  D D D  D D D  D
WER: 58.33%
INFO:tensorflow:
====== WER train_pred ======

REF: Y  UW W AA N T  T UW K  IY P  P IY P AH L AO F Y  AO R  B AE K
HYP: DH AE T S  W AH T    AY W  AA N T  T UW T UW M AY T  AH B AW T
EVA: S  S  S S  S S    D  S  S  S  S S  S S  S S  S S  S  S    S  S
WER: 91.67%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 1.774033, step = 44076 (282.990 sec)
INFO:tensorflow:Saving checkpoints for 44159 into ./model/seq2seq_VF_224_bn/model.ckpt.
INFO:tensorflow:global_step/sec: 0.318126
INFO